# Working with a real world data-set using SQL and Python

## Objectives

*   Understand the dataset for Chicago Public School level performance
*   Store the dataset in an Db2 database on IBM Cloud instance
*   Retrieve metadata about tables and columns and query data from mixed case columns
*   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**:

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR.

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next".

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

The following modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [2]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

  Using cached ibm_db-3.1.0-py3-none-any.whl
  Using cached ibm_db_sa-0.3.3-py3-none-any.whl
  Using cached SQLAlchemy-1.4.36-cp39-cp39-win_amd64.whl (1.6 MB)
  Using cached greenlet-1.1.2-cp39-cp39-win_amd64.whl (101 kB)
  Attempting uninstall: greenlet
    Found existing installation: greenlet 1.1.2
    Uninstalling greenlet-1.1.2:
      Successfully uninstalled greenlet-1.1.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\acer\\anaconda3\\Lib\\site-packages\\~qlalchemy\\cprocessors.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



Found existing installation: SQLAlchemy 1.4.36
Uninstalling SQLAlchemy-1.4.36:
  Successfully uninstalled SQLAlchemy-1.4.36
  Using cached SQLAlchemy-1.3.24-cp39-cp39-win_amd64.whl (1.2 MB)


In [3]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://ysv22163:GAah9UMQ7Kxd2o1Y@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB?security=SSL

'Connected: ysv22163@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [8]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql SELECT TABSCHEMA, TABNAME, CREATE_TIME \
FROM syscat.tables \
where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC'); -- bỏ các schema k cần

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


tabschema,tabname,create_time
YSV22163,PETSALE,2022-05-20 03:57:00.664667
YSV22163,EMPLOYEES,2022-05-20 07:28:50.874070
YSV22163,JOB_HISTORY,2022-05-20 07:28:51.139936
YSV22163,JOBS,2022-05-20 07:28:51.363603
YSV22163,DEPARTMENTS,2022-05-20 07:28:51.580952
YSV22163,LOCATIONS,2022-05-20 07:28:51.787426
YSV22163,PETRESCUE,2022-05-21 11:13:50.158953
YSV22163,INSTRUCTOR,2022-05-23 10:20:13.324005
YSV22163,INTERNATIONAL_STUDENT_TEST_SCORES,2022-05-23 10:43:20.932863
YSV22163,CHICAGO_SOCIOECONOMIC_DATA,2022-05-23 12:27:33.551264


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [9]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql SELECT COUNT(*) FROM syscat.columns WHERE tabname = 'SCHOOLS'

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


1
78


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [11]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql SELECT colname, typename, length \
FROM syscat.columns \
WHERE TABNAME = 'SCHOOLS';

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


colname,typename,length
SCHOOL_ID,INTEGER,4
NAME_OF_SCHOOL,VARCHAR,64
"Elementary, Middle, or High School",VARCHAR,2
STREET_ADDRESS,VARCHAR,29
CITY,VARCHAR,7
STATE,VARCHAR,2
ZIP_CODE,INTEGER,4
PHONE_NUMBER,VARCHAR,14
LINK,VARCHAR,78
NETWORK_MANAGER,VARCHAR,40


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [20]:
# %sql SELECT *FROM SCHOOLS LIMIT 1;
%sql SELECT COUNT(*) FROM schools WHERE "Elementary, Middle, or High School" = 'ES';

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


1
462


Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [25]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT MAX(safety_score) FROM schools;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


1
99


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [26]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT name_of_school, safety_score FROM schools WHERE safety_score = (SELECT MAX(safety_score) FROM schools);

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


name_of_school,safety_score
Abraham Lincoln Elementary School,99
Alexander Graham Bell Elementary School,99
Annie Keller Elementary Gifted Magnet School,99
Augustus H Burley Elementary School,99
Edgar Allan Poe Elementary Classical School,99
Edgebrook Elementary School,99
Ellen Mitchell Elementary School,99
James E McDade Elementary Classical School,99
James G Blaine Elementary School,99
LaSalle Elementary Language Academy,99


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [28]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT name_of_school, average_student_attendance FROM schools ORDER BY average_student_attendance DESC;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


name_of_school,average_student_attendance
Velma F Thomas Early Childhood Center,None
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%
Orozco Fine Arts & Sciences Elementary School,97.60%
Rachel Carson Elementary School,97.60%
Annie Keller Elementary Gifted Magnet School,97.50%
Andrew Jackson Elementary Language Academy,97.40%
Lenart Elementary Regional Gifted Center,97.40%
Disney II Magnet School,97.30%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [29]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT name_of_school, average_student_attendance FROM schools ORDER BY average_student_attendance LIMIT 5;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [30]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT name_of_school, REPLACE(average_student_attendance, '%', '') FROM schools;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


name_of_school,2
Abraham Lincoln Elementary School,96.00
Adam Clayton Powell Paideia Community Academy Elementary School,95.60
Adlai E Stevenson Elementary School,95.70
Agustin Lara Elementary Academy,95.50
Air Force Academy High School,93.30
Albany Park Multicultural Academy,97.00
Albert G Lane Technical High School,96.30
Albert R Sabin Elementary Magnet School,94.70
Alcott High School for the Humanities,92.70
Alessandro Volta Elementary School,96.40


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [33]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT name_of_school, average_student_attendance FROM schools WHERE REPLACE(average_student_attendance, '%', '') < 70;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


name_of_school,average_student_attendance
Barbara Vick Early Childhood & Family Center,60.90%
Chicago Vocational Career Academy High School,68.80%
Dyett High School,62.50%
Manley Career Academy High School,66.80%
Orr Academy High School,66.30%
Richard T Crane Technical Preparatory High School,57.90%
Roberto Clemente Community Academy High School,69.60%
Wendell Phillips Academy High School,63.00%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [38]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT community_area_number, community_area_name, SUM(college_enrollment) FROM schools GROUP BY community_area_number, community_area_name;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


community_area_number,community_area_name,3
1,ROGERS PARK,4068
2,WEST RIDGE,8197
3,UPTOWN,4388
4,LINCOLN SQUARE,4132
5,NORTH CENTER,7541
6,LAKE VIEW,7055
7,LINCOLN PARK,5615
8,NEAR NORTH SIDE,3362
9,EDISON PARK,910
10,NORWOOD PARK,6469


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [40]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT community_area_number, community_area_name, SUM(college_enrollment) AS TOTAL_ENROLLMENT FROM schools GROUP BY community_area_number, community_area_name ORDER BY 3 LIMIT 5;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


community_area_number,community_area_name,total_enrollment
36,OAKLAND,140
37,FULLER PARK,531
47,BURNSIDE,549
76,OHARE,786
32,LOOP,871


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [41]:
# %sql SELECT * FROM SCHOOLS LIMIT 1;
%sql SELECT name_of_school, safety_score FROM schools ORDER BY 2 LIMIT 5;

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


name_of_school,safety_score
Edmond Burke Elementary School,1
Luke O'Toole Elementary School,5
George W Tilton Elementary School,6
Foster Park Elementary School,11
Emil G Hirsch Metropolitan High School,13


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [47]:
# %sql SELECT * FROM CHICAGO_SOCIOECONOMIC_DATA LIMIT 1;
# %sql SELECT TABNAME FROM syscat.tables WHERE TABSCHEMA = 'YSV22163';
%sql SELECT hardship_index \
FROM schools AS sc, CHICAGO_SOCIOECONOMIC_DATA AS ch \
WHERE sc.community_area_number = ch.ca AND college_enrollment = '4368';

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


hardship_index
6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [49]:
# %sql SELECT * FROM CHICAGO_SOCIOECONOMIC_DATA LIMIT 1;
# %sql SELECT TABNAME FROM syscat.tables WHERE TABSCHEMA = 'YSV22163';
%sql SELECT ca, community_area_name, hardship_index \
FROM CHICAGO_SOCIOECONOMIC_DATA \
WHERE ca = (SELECT community_area_number FROM schools ORDER BY college_enrollment DESC LIMIT 1);

 * ibm_db_sa://ysv22163:***@125f9f61-9715-46f9-9399-c8177b21803b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30426/BLUDB
Done.


ca,community_area_name,hardship_index
5.0,North Center,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                        |
| ----------------- | ------- | ----------------- | ----------------------------------------- |
| 2021-07-09        | 2.4     | Malika            | Updated connection string                 |
| 2021-05-19        | 2.3     | Lakshmi Holla     | Updated question                          |
| 2021-04-20        | 2.2     | Malika            | Added the libraries                       |
| 2020-11-27        | 2.1     | Sannareddy Ramesh | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya           | Moved lab to course repo in GitLab        |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
